In [1]:
!pip install torch diffusers transformers accelerate tqdm datasets gdown

In [2]:
import torch
from torch.utils.data import DataLoader
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, PNDMScheduler
from transformers import CLIPTextModel, CLIPTokenizer, CLIPProcessor
from diffusers import DiffusionPipeline
from diffusers.optimization import get_scheduler
# from diffusers.utils import
from datasets import load_dataset
import os
from tqdm import tqdm
from PIL import Image
from torchvision import transforms

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [3]:
# Load the pre-trained Stable Diffusion model and tokenizer
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
pipe.load_lora_weights(repo_id)
_ = pipe.to("cuda")

# Load the specific VAE and replace the default VAE in the pipeline
# vae_name = "madebyollin/sdxl-vae-fp16-fix"
# pipe.vae = torch.load(vae_name)
# pipe.vae.to("cuda")

# Freeze the parameters of the base model if you only want to train the final layers
for param in pipe.unet.parameters():
    param.requires_grad = False

# Replace the unet with the trainable version
pipe.unet = UNet2DConditionModel.from_pretrained(model_name, subfolder="unet")
pipe.text_encoder = CLIPTextModel.from_pretrained(model_name, subfolder="text_encoder")
tokenizer = CLIPTokenizer.from_pretrained(model_name, subfolder="tokenizer")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [4]:
image_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1] range
])

In [5]:
!gdown "1uQs3QuctJU5pkacSNhVKDmBH83s9KdfZ"

Downloading...
From: https://drive.google.com/uc?id=1uQs3QuctJU5pkacSNhVKDmBH83s9KdfZ
To: /workspace/cut.jsonl
100%|████████████████████████████████████████| 446k/446k [00:00<00:00, 5.55MB/s]


In [6]:
!gdown "1WcUb7nMHOfW_wigBUayIXN43R86tXdIf"

Downloading...
From (original): https://drive.google.com/uc?id=1WcUb7nMHOfW_wigBUayIXN43R86tXdIf
From (redirected): https://drive.google.com/uc?id=1WcUb7nMHOfW_wigBUayIXN43R86tXdIf&confirm=t&uuid=071299bf-24b0-46ef-b185-6ba69c349ab4
To: /workspace/cut.zip
100%|██████████████████████████████████████| 29.0M/29.0M [00:01<00:00, 28.0MB/s]


In [7]:
import zipfile
with zipfile.ZipFile("./cut.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, trans):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.trans = trans

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['prompt']
        image_id = "./cut/"+self.dataset[idx]['file_name']
        encoding = self.tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
        # image = Image.open("/content/cut/"+image_id).convert("RGB")
        image = Image.open(image_id).convert("RGB")
        image = self.trans(image)
        encoding['pixel_values'] = image
        encoding['raw'] = text
        return encoding

In [9]:
# Assuming `texts` and `images` are lists of text-image pairs
td = load_dataset("json", data_files="./cut.jsonl", split="train[:100%]")
train_dataset = CustomDataset(td, tokenizer, image_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [10]:
# Training loop
num_epochs = 10

# Define the optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(pipe.unet.parameters(), lr=1e-5)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=500,
    num_training_steps=len(train_dataloader) * num_epochs,
)

In [16]:
for epoch in range(num_epochs):
    pipe.unet.train()
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0.0  # Initialize epoch loss
    
    for step, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to("cuda")
        pixel_values = batch["pixel_values"].to("cuda")
        text = batch["raw"]
        # Forward pass
        optimizer.zero_grad()
        outputs = pipe(prompt=text)
        loss = criterion(outputs, pixel_values)

        # Backward pass
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()  # Accumulate batch loss for epoch
        progress_bar.set_postfix(loss=loss.item())

    # Average epoch loss
    epoch_loss /= len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}")

    # Save the model checkpoint if needed based on validation loss

    # Step learning rate scheduler after each epoch
    lr_scheduler.step()

    # Save the model checkpoint
    pipe.unet.save_pretrained(f"model_checkpoint_{epoch + 1}")

print("Training complete.")

Epoch 1/10:   0%|          | 0/1250 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)